In [1]:
import os
import math
import random
import numpy as np
import re
import dspy
from dspy.teleprompt import *
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

os.environ['TOGETHER_API_KEY'] = '35ba5bebf6288e43fdc8989965161592e3335d7067c772c0c6995cdc0e60cd88'
os.environ['TOGETHER_API_BASE'] = 'https://api.together.xyz/v1'

In [2]:
# constants
NUM_PAIRS = 1
NUM_CITIES = 10
TRAIN_INSTANCES = 100
TEST_INSTANCES = 100
CITIES = " ".join(map(str, list(np.arange(NUM_CITIES))))
# CITIES = "[" + ", ".join(map(str, list(np.arange(NUM_CITIES)))) + "]"
NUM_THREADS = 5
K = 6

In [3]:
# OR Tools
def create_data_model(distance_matrix, constraints):
    """Stores the data for the problem."""
    data = {
        "distance_matrix": distance_matrix,
        "pickups_deliveries": constraints,
        "num_vehicles": 1,
        "depot": 0
    }
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console and returns the route and distance."""
    total_distance = 0
    optimal_route = []
    
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            plan_output += f" {node_index} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        node_index = manager.IndexToNode(index)
        route.append(node_index)
        plan_output += f"{node_index}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        # print(plan_output)
        optimal_route.append(route)
        total_distance += route_distance
    optimal_route = optimal_route[0][:-1]
    # print(f"Total Distance of all routes: {total_distance}m")
    return optimal_route, total_distance


def solve_pdp_with_constraints(locations, constraints, distance_matrix, num_vehicles=1, depot=0):
    """Solve the PDP using OR-Tools."""
    locations = [list(loc) for loc in locations]
    distance_matrix = distance_matrix.astype(int).tolist()

    data = create_data_model(distance_matrix, constraints)

    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), num_vehicles, depot)

    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    for request in data["pickups_deliveries"]:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
        routing.solver().Add(distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index))

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        optimal_route, total_distance = print_solution(data, manager, routing, solution)
        return optimal_route, total_distance
    else:
        print("No solution found!")
        return None

In [4]:
def euclidean_distance(point1, point2):
    return round(math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2), 1)

def calc_path_distance(path, distances):
    total_distance = 0
    if len(path) < 2:
        return 0
    for i in range(len(path) - 1):
        total_distance += distances[path[i]][path[i + 1]]
    total_distance += distances[path[-1]][path[0]]
    return total_distance

def make_graphs(num_instances, num_cities):
    x_range = (-20, 20)
    y_range = (-20, 20)

    distanceList = []
    precedence_constraints = []
    for _ in range(num_instances):
        coordinates = [(random.randint(*x_range), random.randint(*y_range)) for _ in range(num_cities)]
        distance_matrix = [[euclidean_distance(coordinates[i], coordinates[j]) for j in range(num_cities)] for i in range(num_cities)]
        
        # Generate NUM_PAIRS random precedence pairs
        # ensures there is no constraint where 0 is after sumn
        pairs = []
        for _ in range(NUM_PAIRS):
            a, b = random.sample(range(1, num_cities), 2)
            pairs.append((a, b))
        
        distanceList.append(np.array(distance_matrix))
        precedence_constraints.append(pairs)
    return coordinates, distanceList, precedence_constraints

def make_dataset(coordinates, distanceList, precedence_constraints):
    dataset = []
    for i in range(len(distanceList)):
        matrix = distanceList[i]
        constraints = precedence_constraints[i]
        # Replace with your own PDP solver or use OR-tools for a quick setup
        optimal_route, total_distance = solve_pdp_with_constraints(locations=coordinates, constraints=constraints, distance_matrix=matrix)
        data_point = {
            "distance_matrix": matrix.tolist(),
            "route": optimal_route,
            "optimal_distance": total_distance,
            "constraints": constraints
        }
        dataset.append(data_point)
    return dataset

def makeDSPYExamples(dataset):
    exampleList = []
    for example in dataset:
        distances = "[" + ", ".join([f"[{', '.join(map(str, row))}]" for row in example["distance_matrix"]]) + "]"
        route = ", ".join(map(str, example["route"]))
        pickup_node = " ".join([f"{p[0]}" for p in example["constraints"]])
        delivery_node = " ".join([f"{p[1]}" for p in example["constraints"]])
        exampleObj = dspy.Example(cities=CITIES, distances=distances, route=route, pickup=pickup_node, delivery=delivery_node).with_inputs("cities", "distances", "pickup", "delivery")
        exampleList.append(exampleObj)
    return exampleList

def random_baseline(distances, pickup, delivery):
    numbers = list(range(1, NUM_CITIES))
    random.shuffle(numbers)
    numbers.insert(0, 0)
    if numbers.index(pickup) > numbers.index(delivery):
        numbers.remove(pickup)
        numbers.insert(numbers.index(delivery), pickup)
    
    path_length = calc_path_distance(path=numbers, distances=distances)
    return numbers, path_length

In [5]:
# Train set:
train_coordinates, train_dl, train_p = make_graphs(TRAIN_INSTANCES, NUM_CITIES)
train_ds = make_dataset(train_coordinates, train_dl, train_p)
pdp_trainset = makeDSPYExamples(train_ds)

In [6]:
# Test set:
test_coordinates, test_dl, test_p = make_graphs(TEST_INSTANCES, NUM_CITIES)
test_ds = make_dataset(test_coordinates, test_dl, test_p)
pdp_testset = makeDSPYExamples(test_ds)

In [7]:
llama = dspy.Together(model="meta-llama/Meta-Llama-3-70B", max_tokens=50)
dspy.configure(lm=llama)

In [8]:
pdp_trainset[0]

Example({'cities': '0 1 2 3 4 5 6 7 8 9', 'distances': '[[0.0, 9.2, 22.8, 17.7, 19.3, 23.0, 8.1, 23.2, 16.1, 23.5], [9.2, 0.0, 27.1, 19.0, 24.0, 25.7, 5.1, 16.3, 15.7, 31.4], [22.8, 27.1, 0.0, 9.8, 3.6, 4.5, 22.0, 27.8, 13.9, 14.1], [17.7, 19.0, 9.8, 0.0, 8.5, 7.0, 13.9, 18.0, 4.1, 21.1], [19.3, 24.0, 3.6, 8.5, 0.0, 6.1, 19.0, 26.4, 12.2, 13.0], [23.0, 25.7, 4.5, 7.0, 6.1, 0.0, 20.6, 24.2, 11.0, 18.4], [8.1, 5.1, 22.0, 13.9, 19.0, 20.6, 0.0, 15.1, 10.8, 27.3], [23.2, 16.3, 27.8, 18.0, 26.4, 24.2, 15.1, 0.0, 14.2, 38.2], [16.1, 15.7, 13.9, 4.1, 12.2, 11.0, 10.8, 14.2, 0.0, 24.2], [23.5, 31.4, 14.1, 21.1, 13.0, 18.4, 27.3, 38.2, 24.2, 0.0]]', 'route': '0, 1, 6, 7, 8, 3, 5, 2, 4, 9', 'pickup': '3', 'delivery': '2'}) (input_keys={'cities', 'pickup', 'delivery', 'distances'})

In [9]:
def check_order(route, pickup, delivery):
    return route.index(pickup) < route.index(delivery)

class PDP(dspy.Module):
    def __init__(self):
        super().__init__()
        self.make_route = dspy.Predict(PDPSignature)
        
    def forward(self, cities, distances, pickup, delivery):
        pred_route = self.make_route(cities=cities, distances=distances, pickup=pickup, delivery=delivery)
        numbers, pickup, delivery = extract_route(pred_route.route, pickup, delivery)
        dspy.Suggest(
                check_order(numbers, pickup, delivery),
                f"In the route, the index of {pickup} must be before that of {delivery}"
            )
        return pred_route
    
class PDPSignature(dspy.Signature):
    """generate a route starting at city 0 that visits the pickup node before the delivery node, minimizes distance traveled, and visits each city."""
    cities = dspy.InputField(desc="list of city indices to visit")
    distances = dspy.InputField(desc="matrix of distances between the cities")
    route = dspy.OutputField(desc="optimized route visiting all cities and adhering to constraints")
    pickup = dspy.InputField(desc="pickup node")
    delivery = dspy.InputField(desc="delivery node")


def extract_route(route, pickup, delivery, N=NUM_CITIES):
    # Extract the first N numbers from the route string
    numbers = re.findall(r'\d+', route)[:N]
    
    # Convert the numbers to integers
    numbers = list(map(int, numbers))

    pattern = r'\((\d+),\s*(\d+)\)'

    pickup = int(pickup)
    delivery = int(delivery)
    
    return numbers, pickup, delivery

def eval_tour(cities, route, distances,  pickup, delivery):
    distances_matrix = np.array(eval(distances))
    
    try:
        route, pickup, delivery = extract_route(route, pickup, delivery)  # make it a list of ints
    except ValueError:
        raise ValueError(f"Invalid route: {route}")
    if len(route) != len(distances_matrix):
        raise ValueError(f"Route length {len(route)} does not match number of cities {len(distances_matrix)}")

    # Check precedence constraints
    if route.index(pickup) > route.index(delivery):
        raise ValueError(f"Precedence constraint violated: {pickup} must be visited before {delivery}")
    
    total_distance = sum(distances_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
    total_distance += distances_matrix[route[-1]][route[0]]
    return total_distance

# Validation function for the Precedence-Constrained TSP
def metric(example, pred, trace=None):
    try:
        distance = eval_tour(example.cities, pred.route, example.distances, example.pickup, example.delivery)
        return -distance  # Return negative distance to maximize the metric
    except ValueError as e:
        dspy.logger.error(e)
        return -200

In [ ]:
from dspy.primitives.assertions import backtrack_handler

# Transform the module to include the backtracking mechanism
baleen_with_suggestions = PDP().activate_assertions(backtrack_handler)

teleprompter = LabeledFewShot(k=K)
compiled_pdp = teleprompter.compile(baleen_with_suggestions, trainset=pdp_trainset)

evaluater = Evaluate(devset=pdp_testset, metric=metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)
evaluater(compiled_pdp)

Average Metric: -160.79999999999998 / 1  (-16080.0):   1%| | 1/100 [00:02<04:03,2024-08-05T17:15:23.929526Z [error    ] Precedence constraint violated: 1 must be visited before 8 [__main__] filename=1034983783.py lineno=65
Average Metric: -5972.599999999999 / 35  (-17064.6):  35%|▎| 35/100 [01:53<02:59

In [ ]:
llama.inspect_history(n=1)

In [ ]:
test_example = pdp_testset[0]
numerical_test_example = test_ds[0]
print(numerical_test_example)

In [ ]:
predicted_result = compiled_pdp(cities=test_example.cities, distances=test_example.distances, pickup=test_example.pickup, delivery=test_example.delivery)

predicted_route = predicted_result.route

predicted_distance = eval_tour(test_example.cities, predicted_route, test_example.distances, test_example.pickup, test_example.delivery)

print(f"Predicted route: {predicted_route}")
print(f"Total distance of the predicted route: {predicted_distance}")

optimal_route = test_example.route
optimal_distance = eval_tour(test_example.cities, optimal_route, test_example.distances, test_example.pickup, test_example.delivery)
print(f"Optimal route: {optimal_route}")
print(f"Total distance of the optimal route: {optimal_distance}")

In [ ]:
test_example

In [ ]:
print(numerical_test_example)
path, distance = random_baseline(numerical_test_example["distance_matrix"], numerical_test_example["constraints"][0][0],numerical_test_example["constraints"][0][1])
print(f"path is {path}")
print(f"distance is {distance}")

Random baseline eval:

In [ ]:
total_dis = 0
for i in range(TEST_INSTANCES):
    curr_example = test_ds[i]
    _, distance = random_baseline(curr_example["distance_matrix"], curr_example["constraints"][0][0], curr_example["constraints"][0][1])
    total_dis += distance
print(f"(RANDOM) total distance is {total_dis}")
print(f"(RANDOM) average distance is {total_dis/TEST_INSTANCES}")

In [ ]:
zs_TSP = evaluater(PDP())
print(f"(Zero Shot) average distance is {zs_PDP / len(pdp_testset)}")

Model eval:

In [ ]:
print("(MODEL) average distance is 159.6")

Optimal route eval:

In [ ]:
total_dis = 0
for i in range(TEST_INSTANCES):
    curr_example = test_ds[i]
    total_dis += curr_example["optimal_distance"]
print(f"(OPTIMAL) total distance is {total_dis}")
print(f"(OPTIMAL) average distance is {total_dis/TEST_INSTANCES}")